In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import xml.etree.ElementTree as ET
import os

HOME = os.path.abspath(os.sep)
dataset_folder = os.getcwd() + "/Dataset/Images"

In [2]:
import os
import json
import torch
from PIL import Image
from torch.utils.data import Dataset

class GunDataset(Dataset):
    def __init__(self, root_dir, annotations_file, transforms=None):
        """
        Args:
            root_dir (str): Directory with images.
            annotations_file (str): Path to _annotations.coco.json.
            transform (callable, optional): Optional transform.
        """
        with open(annotations_file, 'r') as f:
            coco_data = json.load(f)
            
        self.root_dir = root_dir
        self.transforms = transforms
        self.images = coco_data['images']
        self.annotations = coco_data['annotations']
        self.classes = ['gun']

        self.img_to_anns = {img['id']: [] for img in self.images}
        for ann in self.annotations:
            self.img_to_anns[ann['image_id']].append(ann)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_info = self.images[idx]
        img_id = img_info['id']
        img_path = f"{self.root_dir}/{img_info['file_name']}"
        image = Image.open(img_path).convert("RGB")

        annotations = self.img_to_anns[img_id]
        boxes = []
        labels = []
        iscrowd = []
        areas = []

        for ann in annotations:
            bbox = ann['bbox']
            x_min, y_min, width, height = bbox
            x_max = x_min + width
            y_max = y_min + height
            boxes.append([x_min, y_min, x_max, y_max])
            area = width * height
            areas.append(area)
            label = ann['category_id']
            labels.append(label)
            iscrowd.append(ann.get('iscrowd', 0))

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        iscrowd = torch.tensor(iscrowd, dtype=torch.int64)
        areas = torch.tensor(areas, dtype=torch.float32)

        target = {
            'image_id': img_id,
            'boxes': boxes,
            'labels': labels,
            'iscrowd': iscrowd,
            'area': areas
        }

        if self.transforms:
            image = self.transforms(image)

        return image, target

In [3]:
import torchvision.transforms as T
from torch.utils.data import DataLoader, random_split

transforms = T.Compose([T.ToTensor()])

# load dataset
dataset = GunDataset(dataset_folder, dataset_folder + "/_annotations.coco.json", transforms=transforms)

# define splits
train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

num_workers = 2
if os.name == 'nt':
    num_workers = 0

# data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=num_workers, collate_fn=lambda x: tuple(zip(*x)), pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=num_workers, collate_fn=lambda x: tuple(zip(*x)))

In [4]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [5]:
from engine import train_one_epoch, evaluate
from metrics_utils import MetricsLogger

num_epochs = 20
metrics_logger = MetricsLogger('base')

for epoch in range(num_epochs):
    # Training
    train_metrics = train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)
    
    # Validation
    coco_evaluator = evaluate(model, val_loader, device)
    
    # Update learning rate
    lr_scheduler.step()
    
    # Log metrics
    metrics_logger.log_epoch(
        epoch_num=epoch,
        train_loss=train_metrics.meters['loss'].global_avg,
        lr=optimizer.param_groups[0]['lr'],
        coco_evaluator=coco_evaluator
    )

# Save final metrics
metrics_logger.save_training_metrics()

Epoch: [0]  [  0/297]  eta: 0:08:56  lr: 0.000022  loss: 0.8161 (0.8161)  loss_classifier: 0.6730 (0.6730)  loss_box_reg: 0.1245 (0.1245)  loss_objectness: 0.0112 (0.0112)  loss_rpn_box_reg: 0.0074 (0.0074)  time: 1.8051  data: 0.0421  max mem: 5782
Epoch: [0]  [ 10/297]  eta: 0:04:45  lr: 0.000191  loss: 0.7072 (0.6781)  loss_classifier: 0.5582 (0.5202)  loss_box_reg: 0.1245 (0.1313)  loss_objectness: 0.0135 (0.0179)  loss_rpn_box_reg: 0.0088 (0.0087)  time: 0.9932  data: 0.0199  max mem: 5940
Epoch: [0]  [ 20/297]  eta: 0:04:21  lr: 0.000359  loss: 0.4736 (0.5080)  loss_classifier: 0.2599 (0.3495)  loss_box_reg: 0.1229 (0.1307)  loss_objectness: 0.0148 (0.0184)  loss_rpn_box_reg: 0.0091 (0.0094)  time: 0.9000  data: 0.0174  max mem: 5940
Epoch: [0]  [ 30/297]  eta: 0:04:07  lr: 0.000528  loss: 0.2938 (0.4407)  loss_classifier: 0.1481 (0.2874)  loss_box_reg: 0.1198 (0.1274)  loss_objectness: 0.0152 (0.0170)  loss_rpn_box_reg: 0.0091 (0.0089)  time: 0.8915  data: 0.0168  max mem: 5940


In [6]:
path_model = "Model/faster_rcnn.onnx"
if not os.path.exists("Model"):
    os.makedirs("Model")

input = torch.randn(1, 3, 150, 150)
print(input.shape)
torch.onnx.export(
    model, 
    input.cuda(), 
    path_model, 
    export_params=True, 
    opset_version=17, 
    input_names=['image'], 
    output_names=['boxes', 'labels', 'scores']
)



torch.Size([1, 3, 150, 150])


c:\Users\Gianmarco\anaconda3\Lib\site-packages\torch\nn\functional.py:3982: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (torch.floor((input.size(i + 2).float() * torch.tensor(scale_factors[i], dtype=torch.float32)).float()))
c:\Users\Gianmarco\anaconda3\Lib\site-packages\torchvision\ops\boxes.py:157: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
c:\Users\Gianmarco\anaconda3\Lib\site-packages\torchvision\ops\boxes.py:159: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tenso